# Twitter Data Updates

This Python code will be used to update and append the data sets created in `TwitterData-Start.ipynb` since we can only access the Twitter data of the last 7 days.

In [1]:
# Import Dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import timedelta
import tweepy

from config import consumer_key
from config import consumer_secret
from config import access_token
from config import access_token_secret
from config import token

In [2]:
# API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret
token = token

## Count of Gas & Oil Tweets

In [3]:
query = "gas oil -is:retweet"
client = tweepy.Client(bearer_token=token)
counts = client.get_recent_tweets_count(query=query, 
                                        granularity='day')

In [4]:
Lst = []
# Dict = {}
for i in counts.data:
    Dict = {}
    Dict['Day'] = str(i["start"][0:10])
    Dict['Tweet Count'] = i["tweet_count"]
    Lst.append(Dict)

In [5]:
df = pd.DataFrame(Lst)
df = pd.DataFrame.from_dict(Lst)
df = pd.DataFrame.from_records(Lst)
df = df.rename(columns={"Day": "Date"})
df['Date'] = pd.to_datetime(df['Date'])
# df.to_csv('tweet_count.csv', index=False) 
update_df = df.iloc[-2:-1]

In [6]:
update_df

,Date,Tweet Count
6,2022-12-07,5434


In [7]:
df

,Date,Tweet Count
0,2022-12-01,5135
1,2022-12-02,5183
2,2022-12-03,5081
3,2022-12-04,4638
4,2022-12-05,5986
5,2022-12-06,6572
6,2022-12-07,5434
7,2022-12-08,550


In [8]:
tweet_count = pd.read_csv("./tweet_count.csv")

In [9]:
tweet_count = tweet_count.rename(columns={"d": "Date"})
tweet_count['Date'] = pd.to_datetime(tweet_count['Date'])
tweet_count = pd.concat([tweet_count, update_df], ignore_index = True)

,Unnamed: 0,Date,Tweet Count
0,0.0,2022-11-23,5665
1,1.0,2022-11-24,4852
2,2.0,2022-11-25,4282
3,3.0,2022-11-26,3856
4,4.0,2022-11-27,3898
5,5.0,2022-11-28,5089
6,6.0,2022-11-29,6413
7,7.0,2022-11-30,7249
8,8.0,2022-12-01,5941
9,9.0,2022-12-02,5273


In [10]:
tweet_count = tweet_count.drop(columns=['Unnamed: 0'])
tweet_count.to_csv('tweet_count.csv')
tweet_count

,Date,Tweet Count
0,2022-11-23,5665
1,2022-11-24,4852
2,2022-11-25,4282
3,2022-11-26,3856
4,2022-11-27,3898
5,2022-11-28,5089
6,2022-11-29,6413
7,2022-11-30,7249
8,2022-12-01,5941
9,2022-12-02,5273


In [11]:
tweet_count.index = tweet_count.index.map(str)
tweet_count_dict = tweet_count.to_dict()
tweet_count_dict

{'Date': {'0': Timestamp('2022-11-23 00:00:00'),
  '1': Timestamp('2022-11-24 00:00:00'),
  '2': Timestamp('2022-11-25 00:00:00'),
  '3': Timestamp('2022-11-26 00:00:00'),
  '4': Timestamp('2022-11-27 00:00:00'),
  '5': Timestamp('2022-11-28 00:00:00'),
  '6': Timestamp('2022-11-29 00:00:00'),
  '7': Timestamp('2022-11-30 00:00:00'),
  '8': Timestamp('2022-12-01 00:00:00'),
  '9': Timestamp('2022-12-02 00:00:00'),
  '10': Timestamp('2022-12-03 00:00:00'),
  '11': Timestamp('2022-12-04 00:00:00'),
  '12': Timestamp('2022-12-05 00:00:00'),
  '13': Timestamp('2022-12-06 00:00:00'),
  '14': Timestamp('2022-12-07 00:00:00')},
 'Tweet Count': {'0': 5665,
  '1': 4852,
  '2': 4282,
  '3': 3856,
  '4': 3898,
  '5': 5089,
  '6': 6413,
  '7': 7249,
  '8': 5941,
  '9': 5273,
  '10': 5135,
  '11': 4690,
  '12': 6094,
  '13': 6591,
  '14': 5434}}

## Adding Recent Tweets

In [4]:
# Returns the current local date
day1 = date.today()
day2 = day1 + timedelta(days = 1)
day1 = str(day1)
day2 = str(day2)

In [5]:
print(day1)
print(day2)

2022-12-07
2022-12-08


In [6]:
start = day1 + 'T00:00:00.000Z'
end = day2 + 'T00:00:00.00Z'

tweets_list = []

tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                          tweet_fields=['context_annotations', 'created_at'],
                          start_time=start, 
                          end_time=end,
                          max_results=100).flatten(limit=10000)

for tweet in tweets:
    tweets_list.append(tweet.text)
    
len(tweets_list)

5425

In [16]:
tweet_df = pd.DataFrame(tweets_list)
tweet_df.columns = [day1]
tweet_df = tweet_df.drop_duplicates()
tweet_df.to_csv(day1 + ".csv", index=True) 
len(tweet_df)

5377

In [8]:
all_tweets = pd.read_csv("./tweets.csv")

In [9]:
all_tweets = all_tweets.join(tweet_df, how='outer')

In [10]:
all_tweets = all_tweets.drop(columns=['Unnamed: 0'])
all_tweets

,2022-11-24,2022-11-25,2022-11-26,2022-11-27,2022-11-28,2022-11-29,2022-11-30,2022-12-01,2022-12-02,2022-12-03,2022-12-04,2022-12-05,2022-12-06,2022-12-07
0,@BellaWallerstei They have been paying oil and...,@TrainerGonzo @Belinda71555585 It's that easy ...,@HouseGOP They tried to stick the gas price hi...,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs,"@ChrisMartzWX For well over 40 years, I have p...",@POTUS Except that gas prices were on the rise...,@FinancialReview simply nonsense. Don't let th...,@BBCNews You have a responsibility to explain ...,@GustavoBolivar Es una afrenta para la industr...,@SenateGOP Except the oil and gas companies do...,@BladeoftheS He thinks nurses shouldn’t get a ...,Just When We Thought King Coal Was a Pauper\n#...,@VladiDaddyPutin @VVerocika @sandumaiamd @SecG...,With oil and gas prices out of control consume...
1,@BladeoftheS 3 The major difference between UK...,@davidmweissman Clueless! Right he inherited e...,@AnnieForTruth Yep. I remember when an attenda...,@NikkiHaley I agree use our own oil and coal a...,@WhiteHouse His executive orders on the energy...,@robreiner You really need help. Why are you s...,"@GavinNewsom You could end the gas tax, but yo...",@Marmee33 https://t.co/MeXmoIOgc9,@JDCocchiarella Jack; I can answer that questi...,@GOP And Biden has created more jobs in 2 year...,@POTUS No thanks on your nuclear energy deal d...,Alberta officials withholding list of 100s of ...,@SenateGOP Do Republicans realize that true en...,Occidental Petroleum CEO pushes back against C...
2,¡Oil &amp; Gas Reporter está disponible! https...,@laurenboebert Magic money? You mean from the ...,"@RBReich -\n...and the NATO Supremacists said,...",Nothing is being offered near Utah’s oil and g...,@manuel_rdgn @SEGOB_mx @adan_augusto @calderon...,TCS is a proud sponsor of the Oil &amp; Gatsby...,@POTUS Gas prices are down only because you ar...,@Sadiksmart7 @mohamedamiin2 @Qatari You can't ...,@diegowainstein El colectivo trucho de feminis...,"@SandyHasCandy @TomTSEC And somehow, in your b...","@Julesmerch @ABC Has nothing to do with oil, n...",@GavinNewsom Californians are the only state p...,@mlh_creative @GavinNewsom @POTUS Gas went dow...,@BretBaier Hey Bret why didn’t YOU cover the H...
3,@binnahar85 Europe wants to boycott Qatar? Are...,@thetruthsucks12 @haliwell51 @gmbutts No maybe...,@Princemoye1 I thought its about those that ki...,@Scott19490 @isaiahrmartin Yeah because relian...,rtr: U.S. seeks to limit flaring and methane l...,Should California tax oil profits? Gas spike h...,https://t.co/HEMuIIrFin Oil “Quality News Link...,¡Oil &amp; Gas Reporter está disponible! https...,"Cut off from Russian gas and oil imports, much...",@MichelleSteelCA If only our idiot Prime Minis...,¡Oil &amp; Gas Reporter está disponible! https...,Two energy industry groups filed a lawsuit tod...,@TheKouk Well it isn't impacting on the inflat...,@RachelNotley Not according to studies done by...
4,@TheoFleury14 This was always about $$ and the...,You are full of it. You shut down our clean cr...,The big story. Proof that Westminster has been...,@RepBoebert Lauren Boebert’s income is almost ...,¡Oil &amp; Gas Reporter está disponible! https...,This is a classic example of Washington corrup...,@SenatorLankford You do know that US is the la...,#Gold #Silver and #Crypto update for 12/01/22 ...,¡Oil &amp; Gas Reporter está disponible! https...,PUMP AMERICAN OIL/NG NOW TO SAVE THE US ECONOM...,"Not to be that guy, but, this fossil fuel geop...",@GavinNewsom It's time for California to stop ...,@AntiguaMoses @TonyClimate I've read it\nSmoke...,@JoeBellofatto @weed_skeleton @matt_deplorable...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,NaN,NaN,NaN,NaN,NaN,NaN,Oil Refiners More Than Doubled Their CA Profit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7223,NaN,NaN,NaN,NaN,NaN,NaN,Oil #and #Gas #Jobs in United States\n- Progra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7224,NaN,NaN,NaN,NaN,NaN,NaN,@HouseGOP @SteveScalise @RepBethVanD

In [11]:
all_tweets.to_csv('tweets.csv')

In [12]:
all_tweets = all_tweets.dropna()
all_tweets.to_csv('tweets_clean.csv')

In [13]:
all_tweets

,2022-11-24,2022-11-25,2022-11-26,2022-11-27,2022-11-28,2022-11-29,2022-11-30,2022-12-01,2022-12-02,2022-12-03,2022-12-04,2022-12-05,2022-12-06,2022-12-07
0,@BellaWallerstei They have been paying oil and...,@TrainerGonzo @Belinda71555585 It's that easy ...,@HouseGOP They tried to stick the gas price hi...,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs,"@ChrisMartzWX For well over 40 years, I have p...",@POTUS Except that gas prices were on the rise...,@FinancialReview simply nonsense. Don't let th...,@BBCNews You have a responsibility to explain ...,@GustavoBolivar Es una afrenta para la industr...,@SenateGOP Except the oil and gas companies do...,@BladeoftheS He thinks nurses shouldn’t get a ...,Just When We Thought King Coal Was a Pauper\n#...,@VladiDaddyPutin @VVerocika @sandumaiamd @SecG...,With oil and gas prices out of control consume...
1,@BladeoftheS 3 The major difference between UK...,@davidmweissman Clueless! Right he inherited e...,@AnnieForTruth Yep. I remember when an attenda...,@NikkiHaley I agree use our own oil and coal a...,@WhiteHouse His executive orders on the energy...,@robreiner You really need help. Why are you s...,"@GavinNewsom You could end the gas tax, but yo...",@Marmee33 https://t.co/MeXmoIOgc9,@JDCocchiarella Jack; I can answer that questi...,@GOP And Biden has created more jobs in 2 year...,@POTUS No thanks on your nuclear energy deal d...,Alberta officials withholding list of 100s of ...,@SenateGOP Do Republicans realize that true en...,Occidental Petroleum CEO pushes back against C...
2,¡Oil &amp; Gas Reporter está disponible! https...,@laurenboebert Magic money? You mean from the ...,"@RBReich -\n...and the NATO Supremacists said,...",Nothing is being offered near Utah’s oil and g...,@manuel_rdgn @SEGOB_mx @adan_augusto @calderon...,TCS is a proud sponsor of the Oil &amp; Gatsby...,@POTUS Gas prices are down only because you ar...,@Sadiksmart7 @mohamedamiin2 @Qatari You can't ...,@diegowainstein El colectivo trucho de feminis...,"@SandyHasCandy @TomTSEC And somehow, in your b...","@Julesmerch @ABC Has nothing to do with oil, n...",@GavinNewsom Californians are the only state p...,@mlh_creative @GavinNewsom @POTUS Gas went dow...,@BretBaier Hey Bret why didn’t YOU cover the H...
3,@binnahar85 Europe wants to boycott Qatar? Are...,@thetruthsucks12 @haliwell51 @gmbutts No maybe...,@Princemoye1 I thought its about those that ki...,@Scott19490 @isaiahrmartin Yeah because relian...,rtr: U.S. seeks to limit flaring and methane l...,Should California tax oil profits? Gas spike h...,https://t.co/HEMuIIrFin Oil “Quality News Link...,¡Oil &amp; Gas Reporter está disponible! https...,"Cut off from Russian gas and oil imports, much...",@MichelleSteelCA If only our idiot Prime Minis...,¡Oil &amp; Gas Reporter está disponible! https...,Two energy industry groups filed a lawsuit tod...,@TheKouk Well it isn't impacting on the inflat...,@RachelNotley Not according to studies done by...
4,@TheoFleury14 This was always about $$ and the...,You are full of it. You shut down our clean cr...,The big story. Proof that Westminster has been...,@RepBoebert Lauren Boebert’s income is almost ...,¡Oil &amp; Gas Reporter está disponible! https...,This is a classic example of Washington corrup...,@SenatorLankford You do know that US is the la...,#Gold #Silver and #Crypto update for 12/01/22 ...,¡Oil &amp; Gas Reporter está disponible! https...,PUMP AMERICAN OIL/NG NOW TO SAVE THE US ECONOM...,"Not to be that guy, but, this fossil fuel geop...",@GavinNewsom It's time for California to stop ...,@AntiguaMoses @TonyClimate I've read it\nSmoke...,@JoeBellofatto @weed_skeleton @matt_deplorable...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3810,"@johnredwood We can't frack, have new nuclear ...","@POTUS Moderating ??? Gas is going up again, o...",@OliviaG37717638 And the UK government priorit...,@RachelNotley High prices wouldn’t matter if y...,Sabah should install meter gauge to record oil...,@GBNEWS @LiamHalliga

In [15]:
all_tweets.index = all_tweets.index.map(str)
all_tweets_dict = all_tweets.to_dict()